In [1]:
import csv
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.metrics import precision_score
from sklearn.preprocessing import MinMaxScaler

In [2]:
def file_convert(file_path):
    xt=[]
    yt=[]
    with open(file_path, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            yt.append(int(row[0]))
            #print(yt)
            m=[]
            for i in row[1:]:
                m.append(int(i))
            xt.append(m)
            #print(xt)
    scaler = MinMaxScaler()
    xt = scaler.fit_transform(xt)
    return(xt,yt)

In [3]:
def dis(x,y):
    dis=0
    for i in range(784):
        dis+=(x[i]-y[i])**2
    dis=dis**(0.5)
    return(dis)

In [4]:
q2_train = "C:\\Users\\prana\\OneDrive\\Desktop\\ML3Q2\\train.csv"
xt,yt=file_convert(q2_train)
q2_test="C:\\Users\\prana\\OneDrive\\Desktop\\ML3Q2\\test.csv"
xtest,ytest=file_convert(q2_test)

In [5]:
mlp = MLPClassifier()
mlp.fit(xt,yt)
predictions = mlp.predict(xtest)
accuracy = accuracy_score(ytest, predictions)
print(accuracy)

0.603755868544601


C:\Users\prana\Desktop\New folder\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [6]:
def cluster_assign(k,xt):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(xt)
    cluster_assignments = kmeans.labels_
    d={}
    for i in range(len(xt)):
        if cluster_assignments[i] in d:
            d[cluster_assignments[i]].append(i)
        else:
            d[cluster_assignments[i]]=[i]
    return(d)
    

In [7]:
def center(k,xt):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(xt)
    cluster_assignments = kmeans.labels_
    d=cluster_assign(k,xt)
    geomcenters=kmeans.cluster_centers_
    centers={}
    near_points={}
    for i in range(0,k):
        if i in d:
            k_near_points=[]
            l=[]
            geomcenter=geomcenters[i]
            min=dis(geomcenter,xt[d[i][0]])
            min_pt=d[i][0]
            for j in d[i]:
                a=dis(geomcenter,xt[j])
                l.append((a,j))
                if a<min:
                    min_pt=j
            centers[i]=min_pt
            l.sort()
            r=max(1,len(d[i])*0.2)
            for p in range(int(r)):
                k_near_points.append(l[p][1])
            near_points[i]=k_near_points
            
        
    return(centers,d,near_points)
            

In [8]:
def clustering(xt,yt,xtest,ytest):
    mlp = MLPClassifier()
    mlp.fit(xt,yt)
    predictions = mlp.predict(xtest)
    accuracy = accuracy_score(ytest, predictions)
    print("full data accuracy",accuracy)
    no_of_clusters=[i for i in range(20,101,5)]
    accuracy_with_centroids=[]
    accuracy_with_extrapolated_to_fulldata=[]
    accuracy_with_extrapolated_to_near_points=[]
    accuracy_with_random_points=[]
    precision_with_centroids=[]
    precision_with_extrapolated_to_fulldata=[]
    precision_with_extrapolated_to_near_points=[]
    precision_with_random_points=[]
    for k in range(20,101,5):
        xtrain=xt[:k]
        ytrain=yt[:k]
        mlp = MLPClassifier()
        mlp.fit(xtrain,ytrain)
        predictions = mlp.predict(xtest)
        accuracy = accuracy_score(ytest, predictions)
        accuracy_with_random_points.append(accuracy)
        precision = precision_score(ytest, predictions,average='weighted')
        precision_with_random_points.append(precision)
        centers,d,near_points=center(k,xt)
        xtrain=[]
        ytrain=[]
        for i in range(0,k):
            xtrain.append(xt[centers[i]])
            ytrain.append(yt[centers[i]])
        mlp = MLPClassifier()
        mlp.fit(xtrain,ytrain)
        predictions = mlp.predict(xtest)
        accuracy = accuracy_score(ytest, predictions)
        accuracy_with_centroids.append(accuracy)
        precision = precision_score(ytest, predictions,average='weighted')
        precision_with_centroids.append(precision)
        #d=cluster_assign(k,xt)
        #print(d,centers)
        xtrain=[]
        ytrain=[]
        for i in range(0,k):
            y=yt[centers[i]]
            for j in d[i]:
                xtrain.append(xt[j])
                ytrain.append(y)
        #print(len(xtrain),len(ytrain))
        mlp = MLPClassifier()
        mlp.fit(xtrain,ytrain)
        predictions = mlp.predict(xtest)
        accuracy = accuracy_score(ytest, predictions)
        accuracy_with_extrapolated_to_fulldata.append(accuracy)
        precision = precision_score(ytest, predictions,average='weighted')
        precision_with_extrapolated_to_fulldata.append(precision)
        xtrain=[]
        ytrain=[]
        for i in range(0,k):
            y=yt[centers[i]]
            for j in near_points[i]:
                xtrain.append(xt[j])
                ytrain.append(y)
        #print(len(xtrain),len(ytrain))
        mlp = MLPClassifier()
        mlp.fit(xtrain,ytrain)
        predictions = mlp.predict(xtest)
        accuracy = accuracy_score(ytest, predictions)
        accuracy_with_extrapolated_to_near_points.append(accuracy)
        precision= precision_score(ytest, predictions,average='weighted')
        precision_with_extrapolated_to_near_points.append(precision)
    table={"no_of_points":no_of_clusters,"accuracy_with_random_points":accuracy_with_random_points,"accuracy_with_centroids":accuracy_with_centroids,"accuracy_with_extrapolated_to_fulldata":accuracy_with_extrapolated_to_fulldata,"accuracy_with_extrapolated_to_near_points":accuracy_with_extrapolated_to_near_points}
    table=pd.DataFrame(table)
    p_table={"no_of_points":no_of_clusters,"precision_with_random_points":precision_with_random_points,"precision_with_centroids":precision_with_centroids,"precision_with_extrapolated_to_fulldata":precision_with_extrapolated_to_fulldata,"precision_with_extrapolated_to_near_points":precision_with_extrapolated_to_near_points}
    p_table=pd.DataFrame(p_table)
    print(table)
    #print(p_table)
        

# Q2 

In [9]:
q2_train = "C:\\Users\\prana\\OneDrive\\Desktop\\ML3Q2\\train.csv"
xt,yt=file_convert(q2_train)
q2_test="C:\\Users\\prana\\OneDrive\\Desktop\\ML3Q2\\test.csv"
xtest,ytest=file_convert(q2_test)
clustering(xt,yt,xtest,ytest)

C:\Users\prana\Desktop\New folder\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


full data accuracy 0.6262910798122066


C:\Users\prana\Desktop\New folder\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\prana\Desktop\New folder\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\prana\Desktop\New folder\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\prana\Desktop\New folder\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Prec

    no_of_points  accuracy_with_random_points  accuracy_with_centroids  \
0             20                     0.150235                 0.218779   
1             25                     0.157746                 0.209390   
2             30                     0.145540                 0.211268   
3             35                     0.150235                 0.260094   
4             40                     0.156808                 0.276995   
5             45                     0.159624                 0.244131   
6             50                     0.173709                 0.293897   
7             55                     0.170892                 0.299531   
8             60                     0.201878                 0.299531   
9             65                     0.197183                 0.264789   
10            70                     0.197183                 0.284507   
11            75                     0.213146                 0.271362   
12            80                     0

C:\Users\prana\Desktop\New folder\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## Q1

In [ ]:
# In[ ]:

q2_train = "/Users/mackbookpro/Desktop/python code practices/train.csv"
xt,yt=file_convert(q2_train)
q2_test="/Users/mackbookpro/Desktop/python code practices/test.csv"
xtest,ytest=file_convert(q2_test)
clustering(xt,yt,xtest,ytest)

/Users/mackbookpro/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


full data accuracy 0.889


/Users/mackbookpro/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mackbookpro/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mackbookpro/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/mackbookpro/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimize

   no_of_points  accuracy_with_random_points  accuracy_with_centroids  \
0            20                       0.4611                   0.5677   
1            30                       0.5913                   0.6310   
2            40                       0.6099                   0.5895   
3            50                       0.6288                   0.6548   
4            60                       0.6541                   0.6388   
5            70                       0.7072                   0.6688   

   accuracy_with_extrapolated_to_fulldata  \
0                                  0.5588   
1                                  0.5425   
2                                  0.5507   
3                                  0.6264   
4                                  0.5855   
5                                  0.6135   

   accuracy_with_extrapolated_to_near_points  
0                                     0.5661  
1                                     0.5542  
2                              